In [7]:
import torch
import gcn_source
import lstm_autoencoder_source
import mediapipe as mp
import numpy as np
import cv2
import time

## 검은 화면에 그래프만 남기기 - 영상 전체

In [ ]:
# Mediapipe Holistic 초기화
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# 비디오 파일 범위 처리 (squat_001.mp4부터 squat_100.mp4까지)
for i in range(1, 101):
    # 비디오 파일 경로 생성
    video_file = f'squat_data/squat_{i:03d}.mp4'
    cap = cv2.VideoCapture(video_file)
    
    if not cap.isOpened():
        print(f"Cannot open video {video_file}")
        continue
    
    # 영상 정보 가져오기
    w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # 출력 파일 경로
    out_file = f'squat_processed/squat_graph_{i:03d}.avi'
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    out = cv2.VideoWriter(out_file, fourcc, fps, (w, h))
    
    prev_time = 0
    
    # Mediapipe Holistic 실행
    with mp_holistic.Holistic(
        static_image_mode=True, min_detection_confidence=0.5, model_complexity=2) as holistic:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print(f"End of video {video_file}")
                break
            
            curr_time = time.time()
            
            # 검은 화면 생성
            black_frame = np.zeros((h, w, 3), dtype=np.uint8)
            
            # BGR 이미지를 RGB로 변환 후 처리
            results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            
            # 랜드마크 그리기
            if results.left_hand_landmarks:
                mp_drawing.draw_landmarks(
                    black_frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            if results.right_hand_landmarks:
                mp_drawing.draw_landmarks(
                    black_frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            if results.face_landmarks:
                mp_drawing.draw_landmarks(
                    black_frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    black_frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
            
            # FPS 계산 및 표시
            sec = curr_time - prev_time
            prev_time = curr_time
            fps = 1 / (sec)
            fps_str = "FPS : %0.1f" % fps
            cv2.putText(black_frame, fps_str, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            # 출력 영상 저장 (화면 출력은 없음)
            out.write(black_frame)
        
    cap.release()
    out.release()

cv2.destroyAllWindows()

cv2.waitKey(1)


I0000 00:00:1736993837.918200 51275490 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1
W0000 00:00:1736993837.999181 51286809 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736993838.044440 51286814 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736993838.047510 51286815 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736993838.048110 51286809 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736993838.048246 51286812 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling su

No Source !!!


-1

## 검은 화면에 그래프만 남기기 - 영상 초기 30프레임만

In [ ]:
# 비디오 파일 범위 처리 (squat_001.mp4부터 squat_100.mp4까지)
for i in range(1, 101):
    # 비디오 파일 경로 생성
    video_file = f'squat_data/squat_{i:03d}.mp4'
    cap = cv2.VideoCapture(video_file)
    
    if not cap.isOpened():
        print(f"Cannot open video {video_file}")
        continue
    
    # 영상 정보 가져오기
    w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # 출력 파일 경로
    out_file = f'squat_start/squat_graph_start_{i:03d}.avi'
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    out = cv2.VideoWriter(out_file, fourcc, fps, (w, h))
    
    prev_time = 0
    frame_count = 0  # 프레임 수 카운트
    
    # Mediapipe Holistic 실행
    with mp_holistic.Holistic(
        static_image_mode=True, min_detection_confidence=0.5, model_complexity=2) as holistic:
        while cap.isOpened() and frame_count < 30:  # 첫 30프레임만 처리
            success, image = cap.read()
            if not success:
                print(f"End of video {video_file}")
                break
            
            curr_time = time.time()
            
            # 검은 화면 생성
            black_frame = np.zeros((h, w, 3), dtype=np.uint8)
            
            # BGR 이미지를 RGB로 변환 후 처리
            results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            
            # 랜드마크 그리기
            if results.left_hand_landmarks:
                mp_drawing.draw_landmarks(
                    black_frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            if results.right_hand_landmarks:
                mp_drawing.draw_landmarks(
                    black_frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            if results.face_landmarks:
                mp_drawing.draw_landmarks(
                    black_frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    black_frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
            
            # FPS 계산 및 표시
            sec = curr_time - prev_time
            prev_time = curr_time
            fps = 1 / (sec)
            fps_str = "FPS : %0.1f" % fps
            cv2.putText(black_frame, fps_str, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            # 출력 영상 저장 (화면 출력은 없음)
            out.write(black_frame)
            
            # 프레임 카운트 증가
            frame_count += 1
        
    cap.release()
    out.release()

cv2.destroyAllWindows()
cv2.waitKey(1)

## 웹캠 영상 그래프로 변환

In [10]:
# Mediapipe Pose 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, 
                    model_complexity=1, 
                    enable_segmentation=False, 
                    min_detection_confidence=0.5, 
                    min_tracking_confidence=0.5)

# 비디오 입력 설정
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Pose 추론
    results = pose.process(rgb_frame)
    
    # 검은 화면 생성
    black_frame = np.zeros((480, 640, 3), dtype=np.uint8)  # 480x640 해상도, 검은 배경
    
    # Pose 결과를 검은 화면에 그리기
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            black_frame, 
            results.pose_landmarks, 
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),  # 랜드마크 스타일
            mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)  # 연결선 스타일
        )
    
    # 결과 화면 출력
    cv2.imshow('Pose Graph on Black Background', black_frame)
    cv2.imshow('video',frame)
    if cv2.waitKey(1) == ord('q'):  # ESC 키로 종료
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

I0000 00:00:1736997963.559283 51275490 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1
W0000 00:00:1736997963.695797 51339543 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736997963.719172 51339548 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


-1